In [1]:
import pandas as pd

In [2]:
# Load the saved file
df = pd.read_csv("../data/kmeans_clustered.csv")

df.head()

,subreddit,id,title,body,created,score,comments,full_text,cleaned_text,sentiment_scores,...,anger,disgust,fear,joy,neutral,sadness,surprise,dominant_emotion,cluster,cluster_name
0,PCOD,1o15k9e,Help with pcod routine please 🥺,I'm 23/F and got diagnosed with pcod/pcos in J...,2025-10-08 05:13:30,2,10,Help with pcod routine please 🥺 I'm 23/F and g...,help pcod routine please im f got diagnosed pc...,"{'LABEL_0': 0.2149903029203415, 'LABEL_1': 0.6...",...,0.026252,0.001100,0.005829,0.049896,0.122957,0.661893,0.132073,sadness,5,Lifestyle Tips & Support Journeys
1,PCOD,1o08fbs,PCOD and spotting during Jogging,I always wished to have a body that can do HII...,2025-10-07 03:45:24,3,2,PCOD and spotting during Jogging I always wish...,pcod spotting jogging always wished body hiit ...,"{'LABEL_0': 0.040413208305835724, 'LABEL_1': 0...",...,0.030216,0.002490,0.016535,0.031060,0.084444,0.651861,0.183393,sadness,1,Menstrual Irregularities & Bleeding Issues
2,PCOD,1nzq2u7,Building a better tracking app - am I focused ...,"Hi everyone, I'm a developer who also has PCOD...",2025-10-06 13:49:46,1,4,Building a better tracking app - am I focused ...,building better tracking app focused right pro...,"{'LABEL_0': 0.21276012063026428, 'LABEL_1': 0....",...,0.908543,0.000894,0.001657,0.009122,0.011872,0.051547,0.016366,anger,2,Weight & Motivation Challenges
3,PCOD,1nvdvd2,My client reversed 13 year old pcod,"13 years, tired of trying all the things in ma...",2025-10-01 13:01:52,3,0,"My client reversed 13 year old pcod 13 years, ...",client reversed year old pcod year tired tryin...,"{'LABEL_0': 0.08836515247821808, 'LABEL_1': 0....",...,0.023781,0.001407,0.004636,0.262257,0.476801,0.143135,0.087983,neutral,5,Lifestyle Tips & Support Journeys
4,PCOD,1nv9u10,is it possible to get pregnant with pcod/pcos?,I have been suffering from PCOD for the longes...,2025-10-01 10:32:38,1,1,is it possible to get pregnant with pcod/pcos?...,possible get pregnant pcodpcos suffering pcod ...,"{'LABEL_0': 0.30627697706222534, 'LABEL_1': 0....",...,0.011418,0.000317,0.001642,0.036219,0.002486,0.939009,0.008908,sadness,0,Diagnosis & Fertility Stress


In [3]:
# pip install python-dotenv

In [4]:
# pip install openai

In [5]:
import pandas as pd
from openai import OpenAI
import os
from dotenv import load_dotenv

# ✅ Load .env and initialize API
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ✅ Function to summarize each cluster using GPT
def summarize_cluster_with_gpt(cluster_posts, cluster_num, cluster_name=None):
    text_block = "\n\n".join(cluster_posts[:10])  # Top 10 posts

    prompt = f"""
You are analyzing social media posts from users with PCOD/PCOS. 
These posts belong to the cluster: "{cluster_name or f'Cluster {cluster_num}'}".

Your task:
1. Read the sample posts
2. Identify the central theme
3. Summarize what users are expressing (challenges, emotions, context)
4. Write a short 3–5 sentence summary
5. Suggest a concise theme name (max 5 words)

Posts:
{text_block}

Output Format:
Theme Name: <concise name>
Summary: <3–5 sentence paragraph>
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o",  # or gpt-4-turbo
            messages=[
                {"role": "system", "content": "You are a social media insights analyst specializing in PCOD health narratives."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"[ERROR] Could not summarize cluster {cluster_num}: {e}"

# ✅ Generate Insight Cards
insight_cards = []

for i in sorted(df['cluster'].unique()):
    cluster_posts = df[df['cluster'] == i]['cleaned_text'].tolist()

    # Handle noise separately
    if i == -1:
        cluster_name = "Noise / Miscellaneous Posts"
    else:
        cluster_name = df[df['cluster'] == i]['cluster_name'].iloc[0] if 'cluster_name' in df.columns and not df[df['cluster'] == i].empty else f"Cluster {i}"

    summary = summarize_cluster_with_gpt(cluster_posts, i, cluster_name)

    print(f"\n📌 Insight Card for Cluster {i} ({cluster_name}):\n{summary}\n{'-'*100}")
    insight_cards.append({
        "cluster": i,
        "cluster_name": cluster_name,
        "summary": summary
    })

# ✅ Save summaries as CSV
output_path = "../data/pcod_cluster_insight_cards.csv"
pd.DataFrame(insight_cards).to_csv(output_path, index=False)
print(f"\n✅ Saved insight cards to: {output_path}")


📌 Insight Card for Cluster 0 (Diagnosis & Fertility Stress):
Theme Name: PCOD/PCOS Diagnosis and Fertility Anxiety

Summary: Users diagnosed with PCOD/PCOS express significant stress and confusion surrounding their condition, particularly regarding fertility and managing symptoms. Many feel overwhelmed by conflicting medical advice and express frustration over being predominantly advised to lose weight or take hormonal medication without addressing underlying issues such as hair thinning, fatigue, and mental health impacts. There is a pervasive sense of anxiety over potential infertility and the lack of clear guidance on effective lifestyle changes or treatments. Users seek support and shared experiences from others in similar situations, hoping for practical advice on managing symptoms and improving their overall health.
----------------------------------------------------------------------------------------------------

📌 Insight Card for Cluster 1 (Menstrual Irregularities & Bleedi